## Create a combined catalog of the fine-tuned Grade A arrivals (from Civilini et al. 2023) and the Grade B, C, and D arrivals (Civilini et al. 2021)

### Import libraries

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
from datetime import datetime, timedelta
import time
import sys

### Load catalog of Grade A through D moonquakes

In [ ]:
mqdir = '../catalogs/FC_catalogs/'
mqcat = pd.read_csv(mqdir + 'fc_lspe_dl_catalog.csv')
mqcat.drop(list(mqcat.filter(regex='Unnamed|index')), axis=1, inplace=True)
evids_all = np.unique(mqcat.evid.tolist())

### Load catalog of fine-tuned Grade A arrivals

In [ ]:
cat_gradeA = pd.read_csv(mqdir + 'GradeA_thermal_mq_catalog_final.csv')
cat_gradeA.drop(list(cat_gradeA.filter(regex='Unnamed|index')), axis=1, inplace=True)
evids_A = np.unique(cat_gradeA.evid.tolist())

### Iterate through moonquakes and compile combined catalog

In [ ]:
evids = []
geophones = []
arrival_times = []
grades = []
for v in np.arange(0,len(evids_all)):
    evid = evids_all[v]
    
    # If Grade A event
    if evid in evids_A:
        evtrows = cat_gradeA.loc[cat_gradeA.evid == evid]
        evtrows = evtrows.reset_index()
        # Iterate through rows
        for r in np.arange(0,len(evtrows)):
            evtrow = evtrows.iloc[r]
            # Arrival time
            evttime = datetime.strptime(evtrow.ft_arrival_time, '%Y-%m-%dT%H:%M:%S.%f')
            evttime_str = datetime.strftime(evttime,'%Y-%m-%dT%H:%M:%S.%f')
            # Append info
            evids.append(evid)
            geophones.append(evtrow.geophone)
            arrival_times.append(evttime_str)
            grades.append('A')
        
    # Otherwise:
    else:
        evtrows = mqcat.loc[mqcat.evid == evid]
        evtrows = evtrows.reset_index()
        # Iterate through rows
        for r in np.arange(0,len(evtrows)):
            evtrow = evtrows.iloc[r]
            # Arrival time
            evttime = datetime.strptime(evtrow.abs_time, '%Y-%m-%dT%H:%M:%S')
            evttime_str = datetime.strftime(evttime,'%Y-%m-%dT%H:%M:%S.%f')
            # Append info
            evids.append(evid)
            geophones.append(evtrow.station)
            arrival_times.append(evttime_str)
            grades.append(evtrow.grade)
            
# Combine into one dataframe
d = {'evid':evids, 'geophone':geophones, 'arrival_time':arrival_times, 'grade':grades}
combined_df = pd.DataFrame(data = d)
combined_df.to_csv(mqdir + 'GradeABCD_combined_catalog.csv',index=False)

### Calculate mean detection time for each event

In [ ]:
# Event IDs
evids_all = np.unique(combined_df.evid.tolist())

# Average detections
mean_arrtimes = []
grades = []
for evid in evids_all:
    
    evtrows = combined_df.loc[combined_df.evid == evid].reset_index()
    # Calculate average arrival time across existing detections
    arrival_times = [datetime.strptime(mqdate, '%Y-%m-%dT%H:%M:%S.%f') for mqdate in evtrows.arrival_time.tolist()]
    mintime = min(arrival_times)
    timedeltas = []
    for a in np.arange(0,len(arrival_times)):
        arrtime = arrival_times[a]
        timedeltas.append((arrtime - mintime).total_seconds())
    #
    mean_timedelta = np.mean(timedeltas)
    mean_time = mintime + timedelta(seconds = mean_timedelta)
    mean_time_str = datetime.strftime(mean_time,'%Y-%m-%dT%H:%M:%S.%f')
    mean_arrtimes.append(mean_time_str)
    grades.append(evtrows.grade.tolist()[0])
    
d = {'evid':evids_all, 'mean_arrival_time':mean_arrtimes, 'grade':grades}
mean_df = pd.DataFrame(data = d)
mean_df = mean_df.sort_values(by=['mean_arrival_time']).reset_index()
mean_df = mean_df.drop(columns=['index'])
mean_df.to_csv(mqdir + 'GradeABCD_avg_arrivals_catalog.csv',index=False)